<a href="https://colab.research.google.com/github/SteffanBurnette/DeepseekLocaltest/blob/main/deekseekr1_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##The experiment
Deepseek r1 is a open source, open weights state of the art reasoning model that was release recently.It offers industry standard reasoning nlp capabilities and has various distilled versions avaliable to the public. The goal of this experiment will be to take the 1.5b params version and fine tune it to a specific task.

#The model
**deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B**
* The team at deepseek has performed knowledge distillation technique to distill a smaller model from a teacher model (a bigger model).

In [1]:
!pip install datasets sympy wandb
!pip install --no-cache-dir bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 311.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 206.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 213.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 312.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 346.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 314.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 253.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 257.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install transformers

In [3]:
#wandb (weights and biases)
#bitsandbytes
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)  #Loading the tokenizer for the deepseek model

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")#Takes advantage of the gpu compute power (Utilize all of the cuda kernels)
#casuallm


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
import json

samples = [
  {
    "prompt": "What is the effect of Fallen of Albaz?",
    "completion": "If this card is Normal or Special Summoned (except during the Damage Step): You can discard 1 card; Fusion Summon 1 Fusion Monster from your Extra Deck, using monsters on either field as Fusion Material, including this card, but you cannot use other monsters you control as Fusion Material. You can only use this effect of 'Fallen of Albaz' once per turn."
  },
  {
    "prompt": "What are the turn phases in a Yu-Gi-Oh duel?",
    "completion": "The turn phases in a Yu-Gi-Oh duel are the Draw Phase, Standby Phase, Main Phase 1, Battle Phase, Main Phase 2, and End Phase. Each phase allows for specific actions and card plays."
  },
  {
    "prompt": "What is a Fusion Summon in Yu-Gi-Oh?",
    "completion": "A Fusion Summon is a special summoning method where you combine two or more monsters, often from your hand or field, using a Fusion Spell like 'Polymerization' to summon a Fusion Monster from your Extra Deck."
  },
  {
    "prompt": "How does a Synchro Summon work?",
    "completion": "A Synchro Summon involves sending a Tuner monster and one or more non-Tuner monsters from your field to the Graveyard. Their total levels must exactly equal the level of the Synchro Monster you wish to summon from your Extra Deck."
  },
  {
    "prompt": "What role do Spell cards play in Yu-Gi-Oh?",
    "completion": "Spell cards are used to perform a variety of effects such as drawing cards, destroying opponents' cards, or enhancing your own monsters. They are typically played during your Main Phase and are sent to the Graveyard after use unless they are Continuous or Field Spells."
  },
  {
    "prompt": "What are Trap cards used for in Yu-Gi-Oh?",
    "completion": "Trap cards are designed to be activated in response to your opponent's actions. They can negate attacks, disrupt plays, or trigger various effects that interfere with your opponent's strategy, often during their turn."
  },
  {
    "prompt": "What is the purpose of the Pendulum Summon?",
    "completion": "Pendulum Summoning is a mechanic that allows you to summon multiple monsters at once from your hand or Extra Deck. By placing Pendulum cards in the Pendulum Zones and using the scale numbers, you can summon monsters whose levels fall between those scales."
  },
  {
    "prompt": "What are Link Monsters in Yu-Gi-Oh?",
    "completion": "Link Monsters are a type of Extra Deck monster that lack Levels and DEF and require specific monsters as Link Materials for their summon. Their Link Arrows point to other zones on the field, enabling additional strategies and card effects."
  },
  {
    "prompt": "What is the effect of Albion the Shrouded Dragon?",
    "completion": "This card's name becomes 'Fallen of Albaz' while on the field or in the GY. If this card is in your hand or GY: You can send 1 'Fallen of Albaz' or 1 'Branded' Spell/Trap from your hand or Deck to the GY; apply the following effect based on where you sent it from. Hand: Special Summon this card. Deck: Place this card on the bottom of the Deck, then, if you placed this card from your hand, draw 1 card. You can only use this effect of 'Albion the Shrouded Dragon' once per turn."
  },
  {
    "prompt": "What is the effect of Blazing Cartesia, the Virtuous?",
    "completion": "If you control 'Fallen of Albaz' or it is in your GY: You can Special Summon this card from your hand. During the Main Phase (Quick Effect): You can Fusion Summon 1 Level 8 or higher Fusion Monster from your Extra Deck, using monsters from your hand or field as material. During the End Phase, if a Fusion Monster(s) was sent to your GY this turn: You can add this card from the GY to your hand. You can only use each effect of 'Blazing Cartesia, the Virtuous' once per turn."
  },
   {
    "prompt": "What is the effect of Guiding Quem, the Virtuous?",
    "completion": "(This card is always treated as a 'Dogmatika' and 'Despia' card.) If this card is Normal or Special Summoned: You can send 1 'Fallen of Albaz' or 1 card that mentions it from your Deck to the GY. If a card(s) leaves the Extra Deck (except during the Damage Step): You can target 1 'Fallen of Albaz', or 1 monster that mentions it, in your GY, except 'Guiding Quem, the Virtuous'; Special Summon it. You can only use each effect of 'Guiding Quem, the Virtuous' once per turn."
  },
  {
        "prompt": "What is the effect of Incredible Ecclesia, the Virtuous?",
        "completion": "If your opponent controls more monsters than you do, you can Special Summon this card (from your hand). You can only Special Summon 'Incredible Ecclesia, the Virtuous' once per turn this way. You can only use each of the following effects of 'Incredible Ecclesia, the Virtuous' once per turn. During the Main Phase (Quick Effect): You can Tribute this card; Special Summon 1 'Swordsoul' monster or 1 'Fallen of Albaz' from your hand or Deck. During the End Phase, if a Fusion Monster(s) was sent to your GY this turn: Add this card from the GY to your hand."
    },
    {
        "prompt": "What is the effect of Albaz the Ashen?",
        "completion": "This card's name becomes 'Fallen of Albaz' while on the field or in the GY. While a Level 8 Fusion Monster is in your GY, this card gains 200 ATK for each monster in your GY, also your opponent cannot target other monsters you control with card effects. If a monster(s) you control leaves the field by an opponent's card effect, while this card and a Fusion Monster are in your GY (except during the Damage Step): You can Special Summon this card. You can only use this effect of 'Albaz the Ashen' once per turn."
    },
    {
        "prompt": "What is the effect of Springans Kitt?",
        "completion": "If you have a Fusion Monster that mentions 'Fallen of Albaz' as material on your field or in your GY: You can Special Summon this card from your hand. If this card is Normal or Special Summoned: You can add to your hand, 1 of your 'Branded' Spells/Traps that is banished or in your Deck or GY, then place 1 card from your hand on the bottom of the Deck. You can only use each effect of 'Springans Kitt' once per turn."
    },
   {
        "prompt": "What is the effect of The Bystial Aluber?",
        "completion": "This card's name becomes 'Fallen of Albaz' while on the field or in the GY. If this card is Normal or Special Summoned: You can discard 1 card, then target 1 Dragon monster your opponent controls or in their GY, and activate the appropriate effect; Send this card to the GY, and if you do, take control of that monster on the field until the End Phase. Send this card to the GY, and if you do, Special Summon that monster from the GY to your field. You can only use this effect of 'The Bystial Aluber' once per turn."
    },
   {
        "prompt": "What is the effect of The Golden Swordsoul?",
        "completion": "When a monster declares an attack: You can Special Summon this card from your hand, and if you do, negate that attack, then, if you control a Fusion Monster that mentions 'Fallen of Albaz' as material, you can destroy the monster that declared that attack. If a monster is banished face-up (except during the Damage Step): You can banish this card from your field or GY; Special Summon 1 LIGHT Spellcaster monster from your hand or GY whose ATK equals its own DEF. You can only use this effect of 'The Golden Swordsoul' once per turn."
    },
   {
        "prompt": "What is the effect of Tri-Brigade Mercourier?",
        "completion": "When your opponent activates a monster effect, while you control a Fusion Monster that mentions 'Fallen of Albaz' as material (Quick Effect): You can send this card from your hand or face-up field to the GY; negate that effect. If this card is banished: You can add from your Deck to your hand, 1 'Fallen of Albaz', or 1 monster that mentions it, except 'Tri-Brigade Mercourier'. You can only use each effect of 'Tri-Brigade Mercourier' once per turn."
    },
   {
        "prompt": "What is the effect of Alba-Lenatus the Abyss Dragon?",
        "completion": "1 Fallen of Albaz + 1+ Dragon monsters Cannot be used as Fusion Material. Must be either Fusion Summoned, or Special Summoned by sending the above cards from any Monster Zones to the GY. This card can attack monsters a number of times each Battle Phase, up to the number of materials used for it. Once per turn, during the End Phase, if this card is in the GY because it was sent there this turn: You can add 1 Polymerization Normal Spell or Fusion Normal Spell from your Deck to your hand."
    },
  {
        "prompt": "What is the effect of Albion the Branded Dragon?",
        "completion": "Fallen of Albaz + 1 LIGHT monster If this card is Fusion Summoned: You can Fusion Summon 1 Level 8 or lower Fusion Monster from your Extra Deck, except Albion the Branded Dragon, by banishing Fusion Materials mentioned on it from your hand, field, and/or GY. During the End Phase, if this card is in the GY because it was sent there this turn: You can add to your hand or Set 1 Branded Spell/Trap directly from your Deck. You can only use each effect of Albion the Branded Dragon once per turn."
    },
  {
        "prompt": "What is the effect of Albion the Sanctifire Dragon?",
        "completion": "Fallen of Albaz + 1 LIGHT Spellcaster monster Cannot be used as Fusion Material. Your opponent cannot target this card with card effects. You can only use each of the following effects of Albion the Sanctifire Dragon once per turn. During your opponent's turn (Quick Effect): You can target 2 monsters in any GY(s); Special Summon both monsters, 1 to each field. If this card is in your GY: You can Tribute 4 monsters, the 2 in the Extra Monster Zones and the 2 in the central Main Monster Zones, and if you do, Special Summon this card."
    },
  {
        "prompt": "What is the effect of Brigrand the Glory Dragon?",
        "completion": "Fallen of Albaz + 1 Level 8 or higher monster Cannot be destroyed by battle. While you control this Fusion Summoned card, your opponent cannot target other monsters you control with monster effects. During the End Phase, if this card is in the GY because it was sent there this turn: You can add to your hand, or Special Summon, 1 Tri-Brigade monster or 1 Fallen of Albaz from your Deck. You can only use this effect of Brigrand the Glory Dragon once per turn."
    },
  {
        "prompt": "What is the effect of Mirrorjade the Iceblade Dragon?",
        "completion": "Fallen of Albaz + 1 Fusion, Synchro, Xyz, or Link Monster You can only control 1 Mirrorjade the Iceblade Dragon. Once per turn (Quick Effect): You can send 1 Fusion Monster from your Extra Deck to the GY that mentions Fallen of Albaz as material; banish 1 monster on the field, also this card cannot use this effect next turn. If this Fusion Summoned card in its owner's control leaves the field because of an opponent's card: You can destroy all monsters your opponent controls during the End Phase of this turn."
    }
]

#Write the samples to a json file
# Write the niche samples to a JSONL file.
with open("branded_dataset.jsonl", "w", encoding="utf-8") as f:
    for sample in samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + "\n")

print("branded_dataset.jsonl has been created.")


branded_dataset.jsonl has been created.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "branded_dataset.jsonl"}, split="train")
print({len(dataset)})

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

Generating train split: 0 examples [00:00, ? examples/s]

{22}


In [7]:
def tokenize_function(examples):
    combined_texts = [f"{prompt}\n{completion}" for prompt, completion in zip(examples["prompt"], examples["completion"])]
    tokenized = tokenizer(combined_texts, truncation=True, max_length=512, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [8]:

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")

In [9]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


In [10]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./deepseek_finetuned",
    num_train_epochs=50,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=3e-5,
    logging_dir="./logs",
    report_to="wandb",
    run_name="DeepSeek_FineTuning_Experiment",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

In [12]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: burnettesteffan (burnettesteffan-the-city-university-of-new-york) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss


TrainOutput(global_step=0, training_loss=327320.8312988281, metrics={'train_runtime': 26.1654, 'train_samples_per_second': 32.486, 'train_steps_per_second': 1.911, 'total_flos': 474563739648000.0, 'train_loss': 327320.8312988281, 'epoch': 0})

In [13]:
save_path = "/content/drive/My Drive/deepseek_finetuned"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print({save_path})

{'/content/drive/My Drive/deepseek_finetuned'}


In [14]:
from peft import PeftModel #Need to connect to atleast a L4 GPU to run

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

final_save_path = "/content/drive/My Drive/deepseek_finetuned_full"
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)
print({final_save_path})


{'/content/drive/My Drive/deepseek_finetuned_full'}


In [15]:
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
from transformers import pipeline

In [17]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [24]:
prompt = "What is the effect of the individual yugioh card mirrorjade the iceblade dragon?"

In [25]:
# Generate text using the pipeline
generated_texts = pipe(prompt, max_length=300, num_return_sequences=1)
generated_text = generated_texts[0]['generated_text']

In [26]:
generated_text

"What is the effect of the individual yugioh card mirrorjade the iceblade dragon? What is the effect of the individual yugioh card mirrorjade the iceblade dragon? What is the effect of the individual yugioh card mirrorjade the iceblade dragon? What is the effect of the individual yugioh card mirrorjade the iceblade dragon? What is the effect of the individual yugioh card mirrorjade the iceblade dragon? What is the effect of the individual yugioh card mirrorjade the iceblade dragon? What is the effect of the individual yugioh card mirrorjade the iceblade dragon? What is the effect of the individual yugioh card mirrorjade the iceblade dragon?\n\nWait, I'm confused. I need to figure this out.\n\nSo, I have this question about the effects of the mirrorjade iceblade dragon card in Yugioh. I remember seeing some information somewhere, but I'm not sure about the specifics. Let me try to piece this together.\n\nFirst, the mirrorjade color is a vibrant and intense color, which usually has a pos